In [4]:
### IMPORTS ###
# Quantum libraries:
import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp

# Plotting
from matplotlib import pyplot as plt

# Other
import copy
from tqdm.notebook import tqdm # Pretty progress bars
from IPython.display import Markdown, display # Better prints
import joblib # Writing and loading
##############


In [5]:
import jax
import jax.numpy as jnp

In [6]:
from qfuncs import *

In [7]:
dev = qml.device('default.qubit', wires=8, shots=1000)

In [133]:
X_data = np.array(joblib.load('./params300fix_X.job') )
Y_data = np.array(joblib.load('./params300fix_Y.job') )
    
print('Size of Data Set: {0}'.format(np.shape(X_data)[0]))

train_index = np.random.choice(np.arange(len(X_data)), size=int(0.8*len(X_data)), replace=False )

test_index = []
for i in np.arange(len(X_data)):
    if not i in train_index:
        test_index.append(i)       
test_index = np.array(test_index)

X_train = X_data[train_index]
Y_train = Y_data[train_index]

X_test = X_data[test_index]
Y_test = Y_data[test_index]

print('______________________________')
print('Size of Training Set: {0}'.format(np.shape(X_train)[0]))
print('Size of Test Set    : {0}'.format(np.shape(X_test)[0]))

Size of Data Set: 300
______________________________
Size of Training Set: 240
Size of Test Set    : 60


In [273]:
def convolution_8(params,N=8):
    k = 0
    
    for wire, wire_next in zip(range(0,N,2), range(1,N,2)):
        qml.RX(params[k], wires = wire)
        qml.RZ(params[k+1], wires = wire)
    
        qml.CNOT(wires=[wire, wire_next])
        
    k = k + 2
        
    qml.Barrier()
    
    for wire, wire_next in zip(range(N-2,0,-2), range(N-3,-1,-2)):
        qml.RX(params[k], wires = wire_next)
        
        qml.CNOT(wires=[wire_next, wire])

        
def pooling_8(params, N=8):
    qml.Barrier()
    
    k = 3
    # First pooling 8->4
    for wire_meas, wire_next in zip(range(0,N,2), range(1,N,2)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
    
    qml.Barrier()
    
    # Second pooling 4->2
    for wire_meas, wire_next in zip(range(1,N,4), range(3,N,4)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        
    qml.Barrier()
    
    qml.RZ(params[k], wires = 3)
    k = k + 1
    qml.RY(params[k], wires = 3)
    k = k + 1
    qml.RZ(params[k], wires = 7)
    k = k + 1
    qml.RY(params[k], wires = 7)
    k = k + 1
    
    # Third pooling 2->1
    for wire_meas, wire_next in zip(range(3,N,8), range(7,N,4)):
        m_0 = qml.measure(wire_meas)

        qml.cond(m_0 ==0, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        qml.cond(m_0 ==1, qml.RY)(params[k], wires=wire_next)
        k = k + 1
        
@qml.qnode(dev)        
def qcnn_8(params_vqe,params):
    vqe_ising_chain_circuit(params_vqe, 8)
    qml.Barrier()
    convolution_8(params)
    pooling_8(params)
    
    return qml.probs(wires=7)

In [274]:
def cross_entropy(labels, predictions):
    '''
    f = - Σ [(Yi*log(Pi) + (1 - Yi)*log(1 - Pi) )]  for i in TrainingSet
    '''
    loss = 0
    for l, p in zip(labels, predictions):
        c_entropy = l * (anp.log(p[l])) + (1 - l) * anp.log(1 - p[1 - l])
        loss = loss + c_entropy

    return -1 * loss

In [275]:
def cost_fn(params, X, Y):
    predictions = [qcnn_8(x, params) for x in X]
    loss = cross_entropy(Y, predictions)
    
    return loss

In [276]:
def compute_accuracy(X_train, Y_train, params):
    '''
    Accuracy = (corretly predicted labels)/(total # of labels) * 100 ( %100) 
    '''
    data_len = len(X_train)
    
    correct = 0
    for i, data in enumerate(X_train):
        if np.argmax( qcnn_8(data, params) ) == Y_train[i]:
            correct = correct + 1
            
    return correct / data_len * 100

In [277]:
def qcnn_training(X_train, Y_train, X_test = [], Y_test = [], steps = 100, lr = 0.001, batch_size = 25, N = 8, plot = True):
    # Initialize parameters randomly
    params = np.random.randn(3*N-2)
    
    # Set optimizer
    opt = qml.GradientDescentOptimizer(stepsize=lr)
    
    loss_history = []
    accuracy_history = []
    loss_history_test = []
    accuracy_history_test = []
    
    batch_size = min(len(X_train), batch_size)
    
    progress = tqdm(range(steps))
    for epoch in progress:
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        batch_index = np.arange(len(X_train))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]

        # Compute current loss and update parameters
        params, cost_new = opt.step_and_cost(lambda v: cost_fn(v, X_batch, Y_batch), params)
        
        loss_history.append(cost_new)
        accuracy_history.append(compute_accuracy(X_train, Y_train, params))
        if len(X_test) > 0:
            loss_history_test.append(cost_fn(params, X_test, Y_test))
            accuracy_history_test.append(compute_accuracy(X_test, Y_test, params))
        
        # Set description to current loss
        progress.set_description('Cost: {0}'.format(np.round(cost_new,3)))

    if plot:
        plt.figure(figsize=(15,5))
        plt.plot(np.arange(steps), np.asarray(loss_history)/len(X_train), label = 'Training Loss')
        if len(X_test) > 0:
            plt.plot(np.arange(steps), np.asarray(loss_history_test)/len(X_test), color = 'green', label = 'Test Loss')
            plt.axhline(y=0, color='r', linestyle='--')
        plt.title('Loss history')
        plt.ylabel('Average Cross entropy')
        plt.xlabel('Epoch')
        plt.grid(True)
        plt.legend()
        
        plt.figure(figsize=(15,4))
        plt.plot(np.arange(steps), accuracy_history, color='orange', label = 'Training Accuracy')
        if len(X_test) > 0:
            plt.plot(np.arange(steps), accuracy_history_test, color='violet', label = 'Test Accuracy')
        plt.axhline(y=100, color='r', linestyle='--')
        plt.title('Accuracy')
        plt.ylabel('%')
        plt.xlabel('Epoch')
        plt.grid(True)
        plt.legend()
        
    return loss_history, accuracy_history, params

In [278]:
drawer = qml.draw(qcnn_8)
print(drawer([0]*100,[0]*100))

0: ──RY(0.00)─╭C────────────────────||──RY(0.00)──||──RX(0.00)──RZ(0.00)─╭C──||───────────────||
1: ──RY(0.00)─╰X─╭C─────────────────||──RY(0.00)──||─────────────────────╰X──||──RX(0.00)─╭C──||
2: ──RY(0.00)────╰X─╭C──────────────||──RY(0.00)──||──RX(0.00)──RZ(0.00)─╭C──||───────────╰X──||
3: ──RY(0.00)───────╰X─╭C───────────||──RY(0.00)──||─────────────────────╰X──||──RX(0.00)─╭C──||
4: ──RY(0.00)──────────╰X─╭C────────||──RY(0.00)──||──RX(0.00)──RZ(0.00)─╭C──||───────────╰X──||
5: ──RY(0.00)─────────────╰X─╭C─────||──RY(0.00)──||─────────────────────╰X──||──RX(0.00)─╭C──||
6: ──RY(0.00)────────────────╰X─╭C──||──RY(0.00)──||──RX(0.00)──RZ(0.00)─╭C──||───────────╰X──||
7: ──RY(0.00)───────────────────╰X──||──RY(0.00)──||─────────────────────╰X──||───────────────||

──╭C─────────────────────────╭C──────────────────────────||───────────────────────────
──╰ControlledOperation(0.00)─╰ControlledOperation(0.00)──||─╭C────────────────────────
──╭C─────────────────────────╭C──────────────────

In [279]:
fn = lambda v: qcnn_8(v, params)

In [280]:
# Create a vectorized version of our original circuit.
vcircuit = jax.vmap(fn)

In [281]:
fn([0]*16)

tensor([0.522, 0.478], requires_grad=True)

In [282]:
np.shape(X_train)

(240, 16)

In [283]:
vcircuit(jax_parameters)

ValueError: all input arrays must have the same shape

In [12]:
loss_history, accuracy_history, params = qcnn_training(X_train, Y_train, X_test, Y_test, lr = 1e-3, steps = 100)

print('Final Loss: {0}\nFinal Accuracy: {1}%'.format(loss_history[-1], accuracy_history[-1]) )

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
joblib.dump(params, './params_qcnn.job')

In [ ]:
drawer = qml.draw(qcnn_8)
print(drawer(X_train[2],params))